# Using Cython

We show how to use Cython to accelerate the computation of a cost function and how to avoid some pitfalls.

If you do not care specifically about [Cython](https://cython.org) and just want to make your code faster, prefer [Numba](https://numba.pydata.org) (see the corresponding Numba tutorial for more details), or try to run iminuit in the PyPy interpreter. Numba is more powerful and easier to use, and you don't have to learn the awkward Cython dialect. Cython is a good choice when you have to call into C code from Python, but it is not a good choice to call into C++ code, for this [pybind11](https://pybind11.readthedocs.io/en/stable/) is the ideal choice. Cython does not fully support the C++ language, it was designed for C.

With that disclaimer out of the way, let's see how to use iminuit with a Cython-compiled function.

In [ ]:
# setup of the notebook
%load_ext Cython
from iminuit import Minuit, describe
import numpy as np
import traceback

The following cell is Cython code and will be compiled to machine code behind the scenes.

In [ ]:
%%cython

def cython_func(double x, double y, double z):
    return (x - 1.) ** 2 + (y - 2.) ** 2 + (z - 3.) ** 2 + 1.

Unfortunately, if we try to pass starting values to Minuit via keywords, we get a failure.

In [ ]:
try:
    m = Minuit(cython_func, x=1, y=2, z=3)
except:
    traceback.print_exc()

What happened? `Minuit` uses the `describe` tool which uses introspection to read the function signature, but this failed here. Without that, Minuit does not know how many parameters this function accepts and their names.

Python built-in functions (like `min`) normally do not have a function signature. Functions from cython and swig also do not have one.

There are a few ways to fix this.

- One can pass parameter names explicitly to Minuit, then it works.
- One can use positional arguments.
- One can tell Cython to embed a signature.

In [ ]:
m = Minuit(cython_func, name=("x", "y", "z"), x=0, y=0, z=0)
m.errordef = Minuit.LEAST_SQUARES
m.migrad()

Alternatively, one can use positional arguments without specifying parameter names.

In [ ]:
m = Minuit(cython_func, 0, 0, 0)
m.errordef = Minuit.LEAST_SQUARES
m.migrad()

A nicer solution is to ask Cython to add the missing function signature. This can be achieved with the `embedsignature(true)` decorator.

In [ ]:
%%cython
cimport cython

@cython.embedsignature(True)  # generate a signature that iminuit can extract
def cython_f(double x, double y, double z):
    return (x - 1.) ** 2 + (y - 2.) ** 2 + (z - 3.) ** 2 + 1.

Now it works.

In [ ]:
m = Minuit(cython_f, x=0, y=0, z=0)
m.errordef = Minuit.LEAST_SQUARES
m.migrad()